In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
df=pd.read_csv("/Users/prateekb/Downloads/MyPrograms/Certification/WIP/Machine Learning/Certifications/OnlineNewsPopularity.csv")
pd.options.display.max_columns = None
#url and timedelta are metadata
df=df.drop(['url','timedelta'],axis=1)
df

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,11.0,346.0,0.529052,1.0,0.684783,9.0,7.0,1.0,1.0,4.523121,8.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,671.0,173.125,26900.0,843300.0,374962.500000,2514.742857,4004.342857,3031.115764,11400.0,48000.0,37033.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.025038,0.025001,0.151701,0.025000,0.773260,0.482679,0.141964,0.037572,0.014451,0.722222,0.277778,0.333791,0.100000,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800
39640,12.0,328.0,0.696296,1.0,0.885057,9.0,7.0,3.0,48.0,4.405488,7.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,616.0,184.000,6500.0,843300.0,192985.714286,1664.267857,5470.168651,3411.660830,2100.0,2100.0,2100.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.029349,0.028575,0.231866,0.681635,0.028575,0.564374,0.194249,0.039634,0.009146,0.812500,0.187500,0.3748

In [5]:
#To Dig further lets try to remove multicolinearity using pvalues/VIF
#First lets try VIF
print("Trying VIF")
from statsmodels.stats.outliers_influence import variance_inflation_factor
ndf=df.drop(['shares'],axis=1)
vif = pd.DataFrame()
vif["features"] = ndf.columns
vif["vif_Factor"] = [variance_inflation_factor(ndf.values, i) for i in range(ndf.shape[1])]
Z=vif[~(vif['vif_Factor']<5)]

#Let try to see if we remove columns through VIF we get any improvement
X=df[Z['features']]
Y=df[['shares']]

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.20,random_state=85)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,Y_train)
Y_pred=lr.predict(X_test)

from sklearn.metrics import r2_score
rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)
print(r2_score(Y_test,Y_pred))

#Lets try StandardScaler
print("After Standard Scaler")
from sklearn.preprocessing import StandardScaler
ss=MinMaxScaler()
ss.fit(X_train)
X_train=ss.transform(X_train)
X_test=ss.transform(X_test)

lr=LinearRegression()
lr.fit(X_train,Y_train)
Y_pred=lr.predict(X_test)

from sklearn.metrics import r2_score
rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)
print(r2_score(Y_test,Y_pred))


Trying VIF


/Users/prateekb/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


14086.887644283946
0.013812461128183684
After Standard Scaler
14086.833107327355
0.013820097103460327


In [7]:
#Let try with pvalues now
print("Trying with pvalues")
X=df.drop(['shares'],axis=1)
Y=df[['shares']]

import statsmodels.api as sm
C=X
ncolumns=[]
C=sm.add_constant(C)
model=sm.OLS(df[['shares']],C).fit()
for i,j in model.pvalues.iteritems():
    if(j>.05): #If j>.05 we reject null hypothesis
        #print (i)
        if(i!='const'):
            ncolumns.append(i)
X=df[ncolumns]
Y=df[['shares']]

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.20,random_state=85)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,Y_train)
Y_pred=lr.predict(X_test)

from sklearn.metrics import r2_score
rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)
print(r2_score(Y_test,Y_pred))

#lets try Standard scaler here
print("After Standard Scaler")
from sklearn.preprocessing import StandardScaler
ss=MinMaxScaler()
ss.fit(X_train)
X_train=ss.transform(X_train)
X_test=ss.transform(X_test)

lr=LinearRegression()
lr.fit(X_train,Y_train)
Y_pred=lr.predict(X_test)

from sklearn.metrics import r2_score
rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print(rmse)
print(r2_score(Y_test,Y_pred))
print("This means with current information it is tough to predict number of shares exactly")

Trying with pvalues
14092.618360621911
0.013009911871632807
After Standard Scaler
14092.618360621931
0.013009911871630142
This means with current information it is tough to predict number of shares exactly


In [13]:
corr=df.corr()
print(corr)


                                n_tokens_title  n_tokens_content  \
n_tokens_title                        1.000000          0.018160   
n_tokens_content                      0.018160          1.000000   
n_unique_tokens                      -0.005318         -0.004737   
n_non_stop_words                     -0.004754          0.017512   
n_non_stop_unique_tokens             -0.005420          0.000373   
num_hrefs                            -0.053496          0.423065   
num_self_hrefs                       -0.014856          0.304682   
num_imgs                             -0.008858          0.342600   
num_videos                            0.051460          0.103699   
average_token_length                 -0.071403          0.167789   
num_keywords                         -0.006077          0.072845   
data_channel_is_lifestyle            -0.070815          0.037548   
 data_channel_is_entertainment        0.132791          0.060200   
data_channel_is_bus                  -0.023902  

In [ ]:
X=df[['num_hrefs','num_imgs,num_videos','average_token_length','num_keywords',' data_channel_is_entertainment',
'data_channel_is_tech','data_channel_is_world','kw_max_min','kw_max_min','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg',
'self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','is_weekend,LDA_02',
'global_subjectivity,rate_positive_words,avg_positive_polarity,max_positive_polarity,'avg_negative_polarity'
,'min_negative_polarity','max_negative_polarity','title_subjectivity','title_sentiment_polarity']]